このノートブックでは、`merge-and-filter` 後のゲノムデータを用いて、PRS の計算手順を説明します。

3種類の乳癌 PRS モデルを事例として用います。
- [PGS000004](https://www.pgscatalog.org/score/PGS000004/) モデルに含まれるバリアント数 313個
- [PGS000015](https://www.pgscatalog.org/score/PGS000015/) モデルに含まれるバリアント数 5,218個
- [PGS000332](https://www.pgscatalog.org/score/PGS000332/) モデルに含まれるバリアント数 6,390,808個

比較的バリアント数の少ない PRS モデルだけでなく、バリアント数が非常に多い PRS モデルであっても、大型のサーバPCを使うことなく、手元の macOS PC で計算可能であることを実感していただけたら幸いです。

### Step 1 `hail` など、必要なモジュールを読み込みます

下記のコードを実行してください。 ページ上側のメニューバーにある `実行` ボタンを押下することで、実行することができます。

In [1]:
import hail as hl
hl.init()
from hail.plot import show
from pprint import pprint
hl.plot.output_notebook()

Running on Apache Spark version 3.1.1
SparkUI available at http://192.168.179.2:4043
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.67-a673309b0445
LOGGING: writing to /Users/hacchy/prs-on-hail/hail-20210612-1231-0.2.67-a673309b0445.log


Loading BokehJS ...

### Step 2 ゲノムデータを読み込みます

下記のコードでは、`merge-and-filter` で保存したゲノムデータを読み込みます。

In [2]:
mt = hl.read_matrix_table('genome-data/1kg.JPT.chrAll.filtered.mt')

読み込んだゲノムデータに含まれる研究対象者の人数とバリアントの個数を表示します。

In [3]:
mt.count()

(10860860, 104)

```
(10860860, 104)
```
と表示されました。

これは、次のことを意味します。
- 研究対象者の人数が 104 名
- バリアントの個数が 10,860,860個

読み込んだゲノムデータのバリアント情報として `variantID` が追加済みであることを確認します。

In [4]:
mt.rows().show(5)

+---------------+------------+-----------+-----------+----------+----------+----------+----------+
| locus         | alleles    | rsid      |      qual | filters  |  info.AF | info.MAF |  info.R2 |
+---------------+------------+-----------+-----------+----------+----------+----------+----------+
| locus<GRCh37> | array<str> | str       |   float64 | set<str> |  float64 |  float64 |  float64 |
+---------------+------------+-----------+-----------+----------+----------+----------+----------+
| 1:16141       | ["C","T"]  | "1:16141" | -1.00e+01 | {}       | 1.10e-02 | 1.10e-02 | 3.35e-01 |
| 1:20113       | ["C","T"]  | "1:20113" | -1.00e+01 | {}       | 2.80e-03 | 2.80e-03 | 3.83e-01 |
| 1:51047       | ["A","T"]  | "1:51047" | -1.00e+01 | {}       | 7.74e-03 | 7.74e-03 | 4.09e-01 |
| 1:51049       | ["A","C"]  | "1:51049" | -1.00e+01 | {}       | 7.75e-03 | 7.75e-03 | 4.09e-01 |
| 1:51050       | ["A","T"]  | "1:51050" | -1.00e+01 | {}       | 7.75e-03 | 7.75e-03 | 4.09e-01 |
+---------------+------------+-----------+-----------+----------+----------+----------+----------+

+----------+-----------+
| info.ER2 | variantID |
+----------+-----------+
|  float64 | str       |
+----------+-----------+
|       NA | "1:16141" |
|       NA | "1:20113" |
|       NA | "1:51047" |
|       NA | "1:51049" |
|       NA | "1:51050" |
+----------+-----------+
showing top 5 rows

`variantID` のカラムが表示されていることをご確認ください。

### Step 3-1 PRSモデルを読み込みます (PGS000004)

まずは、比較的バリアント数の少ない PRS モデル (PGS000004) を用います。

下記のコードを実行すると、`prs-models/PGS000004.txt` のデータが読み込まれます。

In [5]:
model_PGS000004 = hl.import_table('prs-models/PGS000004.txt', impute=True, force=True)

2021-06-12 12:40:30 Hail: INFO: Reading table to impute column types
2021-06-12 12:40:31 Hail: INFO: Finished type imputation
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'reference_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)
  Loading field 'allelefrequency_effect' as type float64 (imputed)


下記のコードは、PRSモデルに含まれるバリアントの個数を表示します。

In [6]:
model_PGS000004.count()

313

`313` と表示されました。

これは、PRSモデルに含まれるバリアントの個数が 313 個であることを意味します。

下記のコードは、読み込んだ PRS モデルに `variantID` を追加します。

In [8]:
model_PGS000004 = model_PGS000004.annotate(
    variantID = hl.str(model_PGS000004.chr_name) + ":" + hl.str(model_PGS000004.chr_position) 
) 

`variantID` を追加したのちの最初の 5 行を表示します。

In [9]:
model_PGS000004.show(5)

,,,,,,
chr_name,chr_position,effect_allele,reference_allele,effect_weight,allelefrequency_effect,variantID
int32,int32,str,str,float64,float64,str
1,100880328,"""T""","""A""",3.73e-02,4.10e-01,"""1:100880328"""
1,10566215,"""G""","""A""",-5.86e-02,3.29e-01,"""1:10566215"""
1,110198129,"""C""","""CAAA""",4.58e-02,7.76e-01,"""1:110198129"""
1,114445880,"""A""","""G""",6.21e-02,1.66e-01,"""1:114445880"""
1,118141492,"""C""","""A""",4.52e-02,2.66e-01,"""1:118141492"""


`variantID` のカラムが追加されていることが分かります。

### Step 3-2 ゲノムデータとPRSモデルに共通するバリアントを抽出します (PGS000004)

下記のコードは、PRSモデルのバリアント情報を `variantID` で検索できるようにします。

In [10]:
model_PGS000004 = model_PGS000004.key_by('variantID')

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを抽出します。

In [15]:
mt_match_PGS000004 = mt.annotate_rows(**model_PGS000004[mt.variantID])
mt_match_PGS000004 = mt_match_PGS000004.filter_rows(hl.is_defined(mt_match_PGS000004.effect_weight))

今後のステップの実行時間を短縮するため、抽出したゲノムデータを保存し、再度読み込みます。
下記のコードは、抽出したゲノムデータを `genome-data/1kg.JPT.chrAll.filtered.matched_PGS000004.mt` に保存します。
15分程度の待ち時間が生じます。

In [18]:
mt_match_PGS000004.write('genome-data/1kg.JPT.chrAll.filtered.matched_PGS000004.mt', overwrite=True)

2021-06-12 12:59:26 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 12:59:27 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 13:06:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 13:11:04 Hail: INFO: wrote matrix table with 272 rows and 104 columns in 22 partitions to genome-data/1kg.JPT.chrAll.filtered.matched_PGS000004.mt
    Total size: 53.57 KiB
    * Rows/entries: 52.73 KiB
    * Columns: 858.00 B
    * Globals: 11.00 B
    * Smallest partition: 2 rows (568.00 B)
    * Largest partition:  24 rows (5.22 KiB)


下記のコードは、`genome-data/1kg.JPT.chrAll.filtered.matched_PGS000004.mt` を読み込みます。

In [19]:
mt_match_PGS000004 = hl.read_matrix_table('genome-data/1kg.JPT.chrAll.filtered.matched_PGS000004.mt')

下記のコードは、ゲノムデータとPRSモデルに共通するバリアントを抽出した結果、ユニークな `variantID` が何個あるかをカウントします。

In [20]:
len(dict(mt_match_PGS000004.aggregate_rows(hl.agg.counter(mt_match_PGS000004.variantID))))

265

`265`と表示されました。

この結果から、PGS000004 モデルに含まれるバリアント数は313個でしたが、そのうち265個が `merge-and-filter` 後のゲノムデータに含まれていることが分かります。

### Step 3-3 PRSを計算して保存します (PGS000004)

下記のコードは、ゲノムデータのアリル情報とPRSモデルのアリル情報を比較し、合致しているかをチェックします。

In [21]:
flip_PGS000004 = hl.case().when( 
    (mt_match_PGS000004.effect_allele == mt_match_PGS000004.alleles[0]) 
    & (mt_match_PGS000004.reference_allele == mt_match_PGS000004.alleles[1]), True ).when( 
    (mt_match_PGS000004.effect_allele == mt_match_PGS000004.alleles[1])
    & (mt_match_PGS000004.reference_allele == mt_match_PGS000004.alleles[0]), False ).or_missing()

In [22]:
mt_match_PGS000004 = mt_match_PGS000004.annotate_rows( flip=flip_PGS000004 )

下記のコードは、各々のバリアントについて研究対象者の持っているアリル数（`mt_match_PGS000004.DS`）とバリアントの重み（`mt_match_PGS000004.effect_weight`）を掛け合わせて、ゲノムデータとPRSモデルの共通するバリアントについて足し合わせます。 これにより、PRSを計算することができます。

In [23]:
prs_PGS000004 = hl.agg.sum( hl.float64( mt_match_PGS000004.effect_weight ) * 
                    hl.if_else( mt_match_PGS000004.flip, 
                                2 - mt_match_PGS000004.DS,
                                mt_match_PGS000004.DS) )

In [24]:
mt_match_PGS000004 = mt_match_PGS000004.annotate_cols(prs=prs_PGS000004)

下記のコードは、PRSの値を表示します。

In [25]:
mt_match_PGS000004.cols().show(5)

2021-06-12 13:17:07 Hail: WARN: cols(): Resulting column table is sorted by 'col_key'.
    To preserve matrix table column order, first unkey columns with 'key_cols_by()'


,
s,prs
str,float64
"""TEST_NA18939_TEST_NA18939""",7.62e-02
"""TEST_NA18940_TEST_NA18940""",9.38e-01
"""TEST_NA18941_TEST_NA18941""",5.07e-01
"""TEST_NA18942_TEST_NA18942""",1.43e-01
"""TEST_NA18943_TEST_NA18943""",7.16e-01


下記のコードは、PRSの分布を表示します。

In [26]:
p = hl.plot.histogram(mt_match_PGS000004.prs, title="PRS Histogram", legend="PGS000004", bins=20)
show(p)

<img src="plots/real-data-plot-01.png" width=50% >

下記のコードは、PRSの計算結果を `1kg.JPT.chrAll.filtered.PGS000004.PRS.txt` ファイルに保存します。

In [27]:
mt_match_PGS000004.cols().export('1kg.JPT.chrAll.filtered.PGS000004.PRS.txt')

2021-06-12 13:21:34 Hail: INFO: Coerced sorted dataset
2021-06-12 13:21:35 Hail: INFO: merging 16 files totalling 3.8K...
2021-06-12 13:21:35 Hail: INFO: while writing:
    1kg.JPT.chrAll.filtered.PGS000004.PRS.txt
  merge time: 85.491ms


### Step 4-1 PRSモデルを読み込みます (PGS000015)

続いて、バリアント数がやや多い PRS モデル (PGS000015) を用います。
比較的バリアント数が少ない PRS モデル (PGS000004) とほぼ変わらない待ち時間で計算できることを実感していただけると思います。

下記のコードを実行すると、`prs-models/PGS000015.txt` のデータが読み込まれます。

In [28]:
model_PGS000015 = hl.import_table('prs-models/PGS000015.txt', impute=True, force=True)

2021-06-12 13:26:19 Hail: INFO: Reading table to impute column types
2021-06-12 13:26:20 Hail: INFO: Finished type imputation
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'reference_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)


下記のコードは、PRSモデルに含まれるバリアントの個数を表示します。

In [29]:
model_PGS000015.count()

5218

`5218` と表示されました。

これは、PRSモデルに含まれるバリアントの個数が 5,218 個であることを意味します。

下記のコードは、読み込んだ PRS モデルに `variantID` を追加します。

In [30]:
model_PGS000015 = model_PGS000015.annotate(
    variantID = hl.str(model_PGS000015.chr_name) + ":" + hl.str(model_PGS000015.chr_position) 
) 

`variantID` を追加したのちの最初の 5 行を表示します。

In [31]:
model_PGS000015.show(5)

,,,,,
chr_name,chr_position,effect_allele,reference_allele,effect_weight,variantID
int32,int32,str,str,float64,str
10,123337335,"""A""","""G""",2.38e-01,"""10:123337335"""
16,52599188,"""T""","""C""",2.15e-01,"""16:52599188"""
11,69331418,"""T""","""C""",2.76e-01,"""11:69331418"""
5,56053535,"""C""","""T""",1.73e-01,"""5:56053535"""
2,217920769,"""G""","""T""",1.27e-01,"""2:217920769"""


`variantID` のカラムが追加されていることが分かります。

### Step 4-2 ゲノムデータとPRSモデルに共通するバリアントを抽出します (PGS000015)

下記のコードは、PRSモデルのバリアント情報を `variantID` で検索できるようにします。

In [32]:
model_PGS000015 = model_PGS000015.key_by('variantID')

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを抽出します。

In [33]:
mt_match_PGS000015 = mt.annotate_rows(**model_PGS000015[mt.variantID])
mt_match_PGS000015 = mt_match_PGS000015.filter_rows(hl.is_defined(mt_match_PGS000015.effect_weight))

今後のステップの実行時間を短縮するため、抽出したゲノムデータを保存し、再度読み込みます。 

下記のコードは、抽出したゲノムデータを `genome-data/1kg.JPT.chrAll.filtered.matched_PGS000015.mt` に保存します。
15分程度の待ち時間が生じます。

In [34]:
mt_match_PGS000015.write('genome-data/1kg.JPT.chrAll.filtered.matched_PGS000015.mt', overwrite=True)

2021-06-12 13:31:09 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 13:31:10 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 13:36:00 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 13:38:04 Hail: INFO: wrote matrix table with 3082 rows and 104 columns in 22 partitions to genome-data/1kg.JPT.chrAll.filtered.matched_PGS000015.mt
    Total size: 533.25 KiB
    * Rows/entries: 532.40 KiB
    * Columns: 858.00 B
    * Globals: 11.00 B
    * Smallest partition: 43 rows (8.69 KiB)
    * Largest partition:  265 rows (50.70 KiB)


下記のコードは、`genome-data/1kg.JPT.chrAll.filtered.matched_PGS000015.mt` を読み込みます。

In [35]:
mt_match_PGS000015 = hl.read_matrix_table('genome-data/1kg.JPT.chrAll.filtered.matched_PGS000015.mt')

下記のコードは、ゲノムデータとPRSモデルに共通するバリアントを抽出した結果、ユニークな `variantID` が何個あるかをカウントします。

In [36]:
len(dict(mt_match_PGS000015.aggregate_rows(hl.agg.counter(mt_match_PGS000015.variantID))))

3054

`3054`と表示されました。

この結果から、PGS000015 モデルに含まれるバリアント数は 5,218 個でしたが、そのうち 3,054 個が `merge-and-filter` 後のゲノムデータに含まれていることが分かります。

### Step 4-3 PRSを計算して保存します (PGS000015)

下記のコードは、ゲノムデータのアリル情報とPRSモデルのアリル情報を比較し、合致しているかをチェックします。

In [37]:
flip_PGS000015 = hl.case().when( 
    (mt_match_PGS000015.effect_allele == mt_match_PGS000015.alleles[0]) 
    & (mt_match_PGS000015.reference_allele == mt_match_PGS000015.alleles[1]), True ).when( 
    (mt_match_PGS000015.effect_allele == mt_match_PGS000015.alleles[1])
    & (mt_match_PGS000015.reference_allele == mt_match_PGS000015.alleles[0]), False ).or_missing()

In [38]:
mt_match_PGS000015 = mt_match_PGS000015.annotate_rows( flip=flip_PGS000015 )

下記のコードは、各々のバリアントについて研究対象者の持っているアリル数（`mt_match_PGS000015.DS`）とバリアントの重み（`mt_match_PGS000015.effect_weight`）を掛け合わせて、ゲノムデータとPRSモデルの共通するバリアントについて足し合わせます。
これにより、PRSを計算することができます。

In [39]:
prs_PGS000015 = hl.agg.sum( hl.float64( mt_match_PGS000015.effect_weight ) * 
                    hl.if_else( mt_match_PGS000015.flip, 
                                2 - mt_match_PGS000015.DS,
                                mt_match_PGS000015.DS) )

In [40]:
mt_match_PGS000015 = mt_match_PGS000015.annotate_cols(prs=prs_PGS000015)

下記のコードは、PRSの値を表示します。

In [41]:
mt_match_PGS000015.cols().show(5)

,
s,prs
str,float64
"""TEST_NA18939_TEST_NA18939""",1.51e+02
"""TEST_NA18940_TEST_NA18940""",1.54e+02
"""TEST_NA18941_TEST_NA18941""",1.51e+02
"""TEST_NA18942_TEST_NA18942""",1.50e+02
"""TEST_NA18943_TEST_NA18943""",1.52e+02


下記のコードは、PRSの分布を表示します。

In [42]:
p = hl.plot.histogram(mt_match_PGS000015.prs, title="PRS Histogram", legend="PGS000015", bins=20)
show(p)

<img src="plots/real-data-plot-02.png" width=50% >

下記のコードは、PRSの計算結果を `1kg.JPT.chrAll.filtered.PGS000015.PRS.txt` ファイルに保存します。

In [43]:
mt_match_PGS000015.cols().export('1kg.JPT.chrAll.filtered.PGS000015.PRS.txt')

2021-06-12 13:47:24 Hail: INFO: Coerced sorted dataset
2021-06-12 13:47:26 Hail: INFO: merging 16 files totalling 3.8K...
2021-06-12 13:47:26 Hail: INFO: while writing:
    1kg.JPT.chrAll.filtered.PGS000015.PRS.txt
  merge time: 104.257ms


### Step 5-1 PRSモデルを読み込みます (PGS000332)

最後に、バリアント数がとても多い PRS モデル (PGS000332) を用います。

下記のコードを実行すると、`prs-models/PGS000332.txt` のデータが読み込まれます。

In [44]:
model_PGS000332 = hl.import_table('prs-models/PGS000332.txt', impute=True, force=True)

2021-06-12 13:52:52 Hail: INFO: Reading table to impute column types
2021-06-12 13:53:01 Hail: INFO: Finished type imputation
  Loading field 'rsID' as type str (imputed)
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'reference_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)


下記のコードは、PRSモデルに含まれるバリアントの個数を表示します。

In [45]:
model_PGS000332.count()

6390808

`6390808` と表示されました。

これは、PRSモデルに含まれるバリアントの個数が 6,390,808 個であることを意味します。

下記のコードは、読み込んだ PRS モデルに `variantID` を追加します。

In [46]:
model_PGS000332 = model_PGS000332.annotate(
    variantID = hl.str(model_PGS000332.chr_name) + ":" + hl.str(model_PGS000332.chr_position) 
) 

`variantID` を追加したのちの最初の 5 行を表示します。

In [47]:
model_PGS000332.show(5)

,,,,,,
rsID,chr_name,chr_position,effect_allele,reference_allele,effect_weight,variantID
str,int32,int32,str,str,float64,str
"""rs61769339""",1,662622,"""A""","""G""",-5.40e-06,"""1:662622"""
"""rs12238997""",1,693731,"""G""","""A""",-6.22e-07,"""1:693731"""
"""rs72631875""",1,705882,"""A""","""G""",1.15e-05,"""1:705882"""
"""rs12029736""",1,706368,"""G""","""A""",-2.08e-06,"""1:706368"""
"""rs116030099""",1,722670,"""C""","""T""",-1.60e-05,"""1:722670"""


`variantID` のカラムが追加されていることが分かります。

### Step 5-2 ゲノムデータとPRSモデルに共通するバリアントを抽出します (PGS000332)

下記のコードは、PRSモデルのバリアント情報を `variantID` で検索できるようにします。

In [48]:
model_PGS000332 = model_PGS000332.key_by('variantID')

下記のコードは、ゲノムデータと PRS モデルに共通するバリアントを抽出します。

In [49]:
mt_match_PGS000332 = mt.annotate_rows(**model_PGS000332[mt.variantID])
mt_match_PGS000332 = mt_match_PGS000332.filter_rows(hl.is_defined(mt_match_PGS000332.effect_weight))

今後のステップの実行時間を短縮するため、抽出したゲノムデータを保存し、再度読み込みます。

下記のコードは、抽出したゲノムデータを `genome-data/1kg.JPT.chrAll.filtered.matched_PGS000332.mt` に保存します。
20分程度の待ち時間が生じます。

In [50]:
mt_match_PGS000332.write('genome-data/1kg.JPT.chrAll.filtered.matched_PGS000332.mt', overwrite=True)

2021-06-12 13:57:36 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 13:57:44 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 14:11:37 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-06-12 14:19:08 Hail: INFO: wrote matrix table with 4687515 rows and 104 columns in 22 partitions to genome-data/1kg.JPT.chrAll.filtered.matched_PGS000332.mt
    Total size: 524.53 MiB
    * Rows/entries: 524.52 MiB
    * Columns: 858.00 B
    * Globals: 11.00 B
    * Smallest partition: 67182 rows (8.15 MiB)
    * Largest partition:  387118 rows (42.31 MiB)


下記のコードは、`genome-data/1kg.JPT.chrAll.filtered.matched_PGS000332.mt` を読み込みます。

In [51]:
mt_match_PGS000332 = hl.read_matrix_table('genome-data/1kg.JPT.chrAll.filtered.matched_PGS000332.mt')

`PGS000004` と `PGS000015` では、ユニークな `variantID` が何個あるかをカウントしていましたが、`PGS000332` ではその操作をスキップします。
`16GB` の RAM では足りず、エラーが表示されてしまったためです。

### Step 5-3 PRSを計算して保存します (PGS000332)

下記のコードは、ゲノムデータのアリル情報とPRSモデルのアリル情報を比較し、合致しているかをチェックします。

In [55]:
flip_PGS000332 = hl.case().when( 
    (mt_match_PGS000332.effect_allele == mt_match_PGS000332.alleles[0]) 
    & (mt_match_PGS000332.reference_allele == mt_match_PGS000332.alleles[1]), True ).when( 
    (mt_match_PGS000332.effect_allele == mt_match_PGS000332.alleles[1])
    & (mt_match_PGS000332.reference_allele == mt_match_PGS000332.alleles[0]), False ).or_missing()

In [56]:
mt_match_PGS000332 = mt_match_PGS000332.annotate_rows( flip=flip_PGS000332 )

下記のコードは、各々のバリアントについて研究対象者の持っているアリル数（`mt_match_PGS000332.DS`）とバリアントの重み（`mt_match_PGS000332.effect_weight`）を掛け合わせて、ゲノムデータとPRSモデルの共通するバリアントについて足し合わせます。
これにより、PRSを計算することができます。

In [57]:
prs_PGS000332 = hl.agg.sum( hl.float64( mt_match_PGS000332.effect_weight ) * 
                    hl.if_else( mt_match_PGS000332.flip, 
                                2 - mt_match_PGS000332.DS,
                                mt_match_PGS000332.DS) )

In [58]:
mt_match_PGS000332 = mt_match_PGS000332.annotate_cols(prs=prs_PGS000332)

下記のコードは、PRSの値を表示します。

In [59]:
mt_match_PGS000332.cols().show(5)

,
s,prs
str,float64
"""TEST_NA18939_TEST_NA18939""",-3.66e-02
"""TEST_NA18940_TEST_NA18940""",3.17e-01
"""TEST_NA18941_TEST_NA18941""",2.74e-01
"""TEST_NA18942_TEST_NA18942""",1.14e-01
"""TEST_NA18943_TEST_NA18943""",-2.77e-02


下記のコードは、PRSの分布を表示します。

In [60]:
p = hl.plot.histogram(mt_match_PGS000332.prs, title="PRS Histogram", legend="PGS000332", bins=20)
show(p)

<img src="plots/real-data-plot-03.png" width=50% >

下記のコードは、PRSの計算結果を 1kg.JPT.chrAll.filtered.PGS000332.PRS.txt ファイルに保存します。

In [61]:
mt_match_PGS000332.cols().export('1kg.JPT.chrAll.filtered.PGS000332.PRS.txt')

2021-06-12 14:34:03 Hail: INFO: Coerced sorted dataset
2021-06-12 14:34:03 Hail: INFO: merging 16 files totalling 3.8K...
2021-06-12 14:34:03 Hail: INFO: while writing:
    1kg.JPT.chrAll.filtered.PGS000332.PRS.txt
  merge time: 45.161ms


### Step 6 PRSのスコア値を比較します

3種類の乳癌PRSモデルを用いて、PRSスコア値を計算しました。
このチュートリアルの最後に、3種類のPRSスコア値を比較してみます。

下記のコードは、PRSモデル `PGS000004` を用いて計算した PRS スコア値を読み込みます。

In [62]:
prs_PGS000004 = hl.import_table('1kg.JPT.chrAll.filtered.PGS000004.PRS.txt', impute=True, force=True)

2021-06-12 14:37:03 Hail: INFO: Reading table to impute column types
2021-06-12 14:37:03 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'prs' as type float64 (imputed)


下記のコードは、PRS スコア値を subject ID (変数名 `s`) で検索できるようにします。

In [63]:
prs_PGS000004 = prs_PGS000004.key_by('s')

下記のコードは、PRSモデル `PGS000015` を用いて計算した PRS スコア値を読み込みます。

In [64]:
prs_PGS000015 = hl.import_table('1kg.JPT.chrAll.filtered.PGS000015.PRS.txt', impute=True, force=True)

2021-06-12 14:39:18 Hail: INFO: Reading table to impute column types
2021-06-12 14:39:18 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'prs' as type float64 (imputed)


下記のコードは、PRS スコア値を subject ID (変数名 `s`) で検索できるようにします。

In [66]:
prs_PGS000015 = prs_PGS000015.key_by('s')

下記のコードは、PRSモデル `PGS000332` を用いて計算した PRS スコア値を読み込みます。

In [67]:
prs_PGS000332 = hl.import_table('1kg.JPT.chrAll.filtered.PGS000332.PRS.txt', impute=True, force=True)

2021-06-12 14:41:51 Hail: INFO: Reading table to impute column types
2021-06-12 14:41:51 Hail: INFO: Finished type imputation
  Loading field 's' as type str (imputed)
  Loading field 'prs' as type float64 (imputed)


下記のコードは、PRS スコア値を subject ID (変数名 s) で検索できるようにします。

In [68]:
prs_PGS000332 = prs_PGS000332.key_by('s')

下記のコードは、3種類の PRS スコア値を subject ID (変数名 `s`) で突合し、データマージを行います。

In [74]:
prs_merge = prs_PGS000004.rename({'s':'subjectID', 'prs':'PGS000004'})

In [82]:
prs_merge = prs_merge.annotate(PGS000015 = prs_PGS000015[prs_merge.subjectID].prs)

In [83]:
prs_merge = prs_merge.annotate(PGS000332 = prs_PGS000332[prs_merge.subjectID].prs)

下記のコードは、データマージした結果を表示します。

In [84]:
prs_merge.show(5)

2021-06-12 14:52:55 Hail: INFO: Coerced sorted dataset
2021-06-12 14:52:55 Hail: INFO: Coerced sorted dataset
2021-06-12 14:52:55 Hail: INFO: Coerced sorted dataset


,,,
subjectID,PGS000004,PGS000015,PGS000332
str,float64,float64,float64
"""TEST_NA18939_TEST_NA18939""",7.62e-02,1.51e+02,-3.66e-02
"""TEST_NA18940_TEST_NA18940""",9.38e-01,1.54e+02,3.17e-01
"""TEST_NA18941_TEST_NA18941""",5.07e-01,1.51e+02,2.74e-01
"""TEST_NA18942_TEST_NA18942""",1.43e-01,1.50e+02,1.14e-01
"""TEST_NA18943_TEST_NA18943""",7.16e-01,1.52e+02,-2.77e-02


下記のコードは、`PGS000004` と `PGS000015` のスコア値をプロットします。

In [92]:
p = hl.plot.scatter(prs_merge.PGS000004, prs_merge.PGS000015, xlabel="PGS000004", ylabel="PGS000015")
show(p)

2021-06-12 14:58:51 Hail: INFO: Coerced sorted dataset
2021-06-12 14:58:51 Hail: INFO: Coerced sorted dataset


<img src="plots/real-data-plot-04.png" width=50% >

このプロットから、`PGS000004` を用いて計算される PRS スコア値 (X軸) と、`PGS000015` を用いて計算される PRS スコア値 (Y軸) は、ある程度相関していることが分かります。

下記のコードは、`PGS000004` と `PGS000332` のスコア値をプロットします。

In [113]:
p = hl.plot.scatter(prs_merge.PGS000004, prs_merge.PGS000332, xlabel="PGS000004", ylabel="PGS000332")
show(p)

2021-06-12 15:23:55 Hail: INFO: Coerced sorted dataset
2021-06-12 15:23:55 Hail: INFO: Coerced sorted dataset


<img src="plots/real-data-plot-05.png" width=50% >

このプロットから、`PGS000004` を用いて計算される PRS スコア値 (X軸) と、`PGS000332` を用いて計算される PRS スコア値 (Y軸) は、ある程度相関していることが分かります。

下記のコードは、`PGS000015` と `PGS000332` のスコア値をプロットします。

In [114]:
p = hl.plot.scatter(prs_merge.PGS000015, prs_merge.PGS000332, xlabel="PGS000015", ylabel="PGS000332")
show(p)

2021-06-12 15:27:07 Hail: INFO: Coerced sorted dataset
2021-06-12 15:27:07 Hail: INFO: Coerced sorted dataset
2021-06-12 15:27:07 Hail: INFO: Coerced sorted dataset


<img src="plots/real-data-plot-06.png" width=50% >

このプロットから、`PGS000015` を用いて計算される PRS スコア値 (X軸) と、`PGS000332` を用いて計算される PRS スコア値 (Y軸) は、ある程度相関していることが分かります。

下記のコードは、PRS スコア値を相関係数を計算します。

`to_pandas()`関数を用いることで、`hail`特有のデータタイプから、`python` でよく使われる `pandas` ライブラリのデータフレームへと変換することができます。
そうすることで、`hail`の関数だけでなく、`python` の様々な機能を使って分析することが可能になります。

In [109]:
prs_merge_pandas = prs_merge.to_pandas()

2021-06-12 15:18:02 Hail: INFO: Coerced sorted dataset
2021-06-12 15:18:02 Hail: INFO: Coerced sorted dataset
2021-06-12 15:18:02 Hail: INFO: Coerced sorted dataset


In [112]:
print(prs_merge_pandas.corr())

           PGS000004  PGS000015  PGS000332
PGS000004   1.000000   0.573695   0.620881
PGS000015   0.573695   1.000000   0.548252
PGS000332   0.620881   0.548252   1.000000


この結果から、次のことが分かります。

- `PGS000004`のスコア値と `PGS000015`のスコア値の相関係数　0.573695
- `PGS000004`のスコア値と `PGS000332`のスコア値の相関係数　0.620881
- `PGS000015`のスコア値と `PGS000332`のスコア値の相関係数　0.548252

3種類の PRS モデルは、モデルに含まれるバリアントの個数は大きく異なりますが、計算される PRS スコア値はある程度相関していました。

### 以上でこのチュートリアルは終了です